# Diabetes Database Data Prep

This notebook is utilized to transform the generalized health risk data from 2011 and 2015 BRFSS survey into a database about diabetes prevalence and potentially related risk factors. 

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
BRFSS_2015 = "2015.csv"
BRFSS_2011 = "2011.csv"
fields = "fields_to_keep.csv"

# Read the Data and store into Pandas DataFrames
df_15 = pd.read_csv(BRFSS_2015)
df_11 = pd.read_csv(BRFSS_2011)
fields_df = pd.read_csv(fields)

In [2]:
# Check the shape of the 2015 data
df_15.shape

(441456, 330)

In [3]:
# Check the shape of the 2011 data
df_11.shape

(506467, 454)

In [4]:
# Confirm that the field list of BRFSS survey fields selected for the diabtes database is loaded as expected
fields_df.head()

,2011,2015
0,_STATE,_STATE
1,IYEAR,IYEAR
2,GENHLTH,GENHLTH
3,PHYSHLTH,PHYSHLTH
4,MENTHLTH,MENTHLTH


In [5]:
# Create lists of fields to keep for 2011 and 2015 data.  This field list was developed by reviewing the BRFSS codebooks for the 2011 and 2015 surveys
# and selecting those fields which reasonably could have a relationship to a health outcome of diabetes.  The column 'DIABETE3' is the column of the
# survey where participants indicate whether they have ever been told they have diabetes.
fields_15 = fields_df['2015'].tolist()
fields_11 = fields_df['2011'].tolist()

In [6]:
# Create Diabetes table by keeping only the needed columns.  This table will be stored in our database and used in our machine learning models.

# First create diabetes table from the 2015 dataframe
diabetes = df_15[fields_15]
diabetes.shape


(441456, 51)

In [7]:

# Update 2015 field names to match 2011 field names.  This is required because, which the questions and responses were the same in both the 
# 2011 and 2015 BRFSS surveys, the field labels are different.  We want one consistent data set in our database, so that names used in 2011 will be
# utilized.

diabetes.rename(columns={'CHCCOPD1':'CHCCOPD','EMPLOY1':'EMPLOY','SCNTMNY1':'SCNTMONY','SCNTMEL1':'SCNTMEAL','_RACE':'RACE2'}, inplace=True)

diabetes.head()


/var/folders/33/_fv8x5t95s93_xt9m7_wzfqc0000gn/T/ipykernel_90511/1774282163.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes.rename(columns={'CHCCOPD1':'CHCCOPD','EMPLOY1':'EMPLOY','SCNTMNY1':'SCNTMONY','SCNTMEL1':'SCNTMEAL','_RACE':'RACE2'}, inplace=True)


,_STATE,IYEAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,...,ADMOVE,MISTMNT,ADANXEV,RACE2,_AGE_G,HTM4,WTKG3,_BMI5,_FRUTSUM,_VEGESUM
0,1.0,b'2015',5.0,15.0,18.0,10.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,1.0,5.0,178.0,12701.0,4018.0,50.0,217.0
1,1.0,b'2015',3.0,88.0,88.0,NaN,2.0,1.0,1.0,4.0,...,NaN,NaN,NaN,1.0,4.0,173.0,7484.0,2509.0,24.0,78.0
2,1.0,b'2015',4.0,15.0,88.0,88.0,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,1.0,6.0,180.0,7167.0,2204.0,NaN,NaN
3,1.0,b'2015',5.0,30.0,30.0,30.0,1.0,2.0,1.0,1.0,...,NaN,NaN,NaN,1.0,5.0,170.0,8165.0,2819.0,100.0,20.0
4,1.0,b'2015',5.0,20.0,88.0,30.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,1.0,5.0,163.0,6441.0,2437.0,NaN,200.0


In [8]:
# Append 2011 data to the diabetes table
diabetes = pd.concat([diabetes,df_11[fields_11]])
diabetes.shape

(947923, 51)

In [9]:
# Check diabetes column values
diabetes['DIABETE3'].value_counts()

3.0    801166
1.0    119717
4.0     15869
2.0      8436
7.0      1200
9.0       402
Name: DIABETE3, dtype: int64

In [10]:
# For our final Diabetes database, we are interested in having data associated with those survey respondents who either do have diabetes or prediabetes
# or those who do not.  From the BRFSS code book, response of:
# '1' means diabetes
# '4' means pre-diabetes
# '3' means no diabetes
# '2' means only had diabetes during pregnancy
# '7' means don't know; '9' means refused to anser and 'BLANK' means the question was not asked.
# We will cleanse our data set to only those records of interest for our machine learning study.

diabetes['DIABETE3'] = diabetes['DIABETE3'].replace({2:0, 3:0})
diabetes['DIABETE3'].value_counts()

0.0    809602
1.0    119717
4.0     15869
7.0      1200
9.0       402
Name: DIABETE3, dtype: int64

In [11]:
# Remove records where respondents answered '7' don't know, or '9' refused.
diabetes = diabetes[diabetes.DIABETE3 != 7]
diabetes = diabetes[diabetes.DIABETE3 != 9]
diabetes['DIABETE3'].value_counts()

0.0    809602
1.0    119717
4.0     15869
Name: DIABETE3, dtype: int64

In [12]:
# Now we want to remove all rows of the dataframe where DIABETE3 has null values; these rows are not relevant for our study
diabetes = diabetes[diabetes['DIABETE3'].notna()]
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 945188 entries, 0 to 506466
Data columns (total 51 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   _STATE    945188 non-null  float64
 1   IYEAR     945188 non-null  object 
 2   GENHLTH   945184 non-null  float64
 3   PHYSHLTH  945187 non-null  float64
 4   MENTHLTH  945188 non-null  float64
 5   POORHLTH  489737 non-null  float64
 6   HLTHPLN1  945188 non-null  float64
 7   PERSDOC2  945188 non-null  float64
 8   MEDCOST   945187 non-null  float64
 9   CHECKUP1  945187 non-null  float64
 10  BPHIGH4   945182 non-null  float64
 11  TOLDHI2   816111 non-null  float64
 12  CVDINFR4  945187 non-null  float64
 13  CVDCRHD4  945187 non-null  float64
 14  CVDSTRK3  945188 non-null  float64
 15  ASTHMA3   945188 non-null  float64
 16  ASTHNOW   124240 non-null  float64
 17  CHCCOPD   945186 non-null  float64
 18  HAVARTH3  945186 non-null  float64
 19  ADDEPEV2  945178 non-null  float64
 20  CHCK

In [13]:
# Note that the IYEAR column datatype is object.  Check the column values
diabetes['IYEAR'].value_counts()

b'2011'    501479
b'2015'    429764
b'2016'     10894
b'2012'      3051
Name: IYEAR, dtype: int64

In [14]:
# Update the values to the expected numerical year values
diabetes['IYEAR'] = diabetes['IYEAR'].replace({"b'2011'":2011,"b'2012'":2011,"b'2015'":2015,"b'2016'":2015 })
diabetes['IYEAR'].value_counts()

2011    504530
2015    440658
Name: IYEAR, dtype: int64

In [15]:
# Confirm the datatype for IYEAR
diabetes.dtypes

_STATE      float64
IYEAR         int64
GENHLTH     float64
PHYSHLTH    float64
MENTHLTH    float64
POORHLTH    float64
HLTHPLN1    float64
PERSDOC2    float64
MEDCOST     float64
CHECKUP1    float64
BPHIGH4     float64
TOLDHI2     float64
CVDINFR4    float64
CVDCRHD4    float64
CVDSTRK3    float64
ASTHMA3     float64
ASTHNOW     float64
CHCCOPD     float64
HAVARTH3    float64
ADDEPEV2    float64
CHCKIDNY    float64
DIABETE3    float64
SEX         float64
MARITAL     float64
EDUCA       float64
RENTHOM1    float64
EMPLOY      float64
SMOKE100    float64
SMOKDAY2    float64
USENOW3     float64
AVEDRNK2    float64
EXERANY2    float64
SCNTMONY    float64
SCNTMEAL    float64
ADPLEASR    float64
ADDOWN      float64
ADSLEEP     float64
ADENERGY    float64
ADEAT1      float64
ADFAIL      float64
ADTHINK     float64
ADMOVE      float64
MISTMNT     float64
ADANXEV     float64
RACE2       float64
_AGE_G      float64
HTM4        float64
WTKG3       float64
_BMI5       float64
_FRUTSUM    float64


In [16]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://diabetesdb.cwyccvgtauvd.us-east-2.rds.amazonaws.com:5432/Diabetes_db"
config = {"user":"postgress", 
          "password": "CWRUDataViz", 
          "driver":"org.postgresql.Driver"}

In [17]:
# Export data to load into SQL DB
diabetes.write.jdbc(url=jdbc_url, table='diabetes', mode=mode, properties=config)

AttributeError: 'DataFrame' object has no attribute 'write'

In [18]:
# Export data to load into SQL DB
diabetes.to_csv("diabetes.csv", index=False)